<a href="https://colab.research.google.com/github/hyeonseong0917/NLP_study/blob/main/Korean_Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
dataset = dataset.data

In [ ]:
dataset[0]

In [ ]:
print(f"데이터셋 총 개수 : {len(dataset)}")

In [ ]:
news_df = pd.DataFrame({'document':dataset})
news_df

In [ ]:
# 먼저 데이터셋 내에 결측값이 있는지 확인합니다
news_df.replace("", float("NaN"), inplace=True)
print(news_df.isnull().values.any())

In [ ]:
news_df = news_df.dropna().reset_index(drop=True)
print(f"필터링된 데이터셋 총 개수 : {len(news_df)}")

In [ ]:
news_df

In [ ]:
# 열을 기준으로 중복제거
processed_news_df = news_df.drop_duplicates(['document']).reset_index(drop=True)
processed_news_df

In [ ]:
len(processed_news_df.iloc[0][0])

In [ ]:
processed_news_df.iloc[0][0]

In [ ]:
# 데이터셋 내 특수 문자를 제거합니다.
processed_news_df['document'] = processed_news_df['document'].str.replace("[^a-zA-Z]", " ")
processed_news_df

In [ ]:
# 문서 내 길이가 너무 짧은 단어를 제거합니다. (단어의 길이가 2 이하)
processed_news_df['document'] = processed_news_df['document'].apply(lambda x: ' '.join([token for token in x.split() if len(token) > 2]))
processed_news_df

In [ ]:
# 전체 길이가 200 이하이거나 전체 단어 개수가 5개 이하인 데이터를 필터링합니다.
processed_news_df = processed_news_df[processed_news_df.document.apply(lambda x: len(str(x)) <= 200 and len(str(x).split()) > 5)].reset_index(drop=True)
processed_news_df

In [ ]:
# 전체 단어에 대한 소문자 변환 (정규화)
processed_news_df['document'] = processed_news_df['document'].apply(lambda x: x.lower())
processed_news_df

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = stopwords.words('english')

print(len(stop_words))
print(stop_words[:10])

In [ ]:
# 데이터들의 불용어를 제외하여 띄어쓰기 단위로 문장을 분리합니다.
tokenized_doc = processed_news_df['document'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [s_word for s_word in x if s_word not in stop_words])
tokenized_doc

In [ ]:
tokenized_doc = tokenized_doc.to_list()
print(len(tokenized_doc))

In [ ]:
print(f"최종 학습 데이터셋 수 : {len(tokenized_doc)}")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

In [ ]:
word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)

In [ ]:
vocab_size = len(word2idx) + 1 
print(f"단어 사전의 크기 : {vocab_size}")

In [ ]:
print(encoded[0])

In [ ]:
from tensorflow.keras.preprocessing.sequence import skipgrams

In [ ]:
# 네거티브 샘플링
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:5]]
print(f"전체 샘플 수 : {len(skip_grams)}")

In [ ]:
# 첫번째 샘플인 skip_grams[0] 내 skipgrams로 형성된 데이터셋 확인
pairs, labels = skip_grams[0][0], skip_grams[0][1]

In [ ]:
print(f"first 3 pairs: {pairs[:3]}")
print(f"first 3 labels: {labels[:3]}")

In [ ]:
# 첫번째 뉴스그룹 샘플에 대해서 생긴 pairs와 labels의 개수
print(len(pairs))
print(len(labels))

In [ ]:
for i in range(5):
  print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
    idx2word[pairs[i][0]], pairs[i][0], 
    idx2word[pairs[i][1]], pairs[i][1], 
    labels[i])
  )

In [ ]:
training_dataset = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:1000]]
print(training_dataset[0][0])

In [ ]:
print(len(training_dataset))

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

In [ ]:
embedding_dim = 100

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(vocab_size, embedding_dim)(c_inputs)

In [ ]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])

dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
print(dot_product)

In [ ]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
print(dot_product)
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
print(dot_product)
output = Activation('sigmoid')(dot_product)
# print(output[0][0])
import keras.backend as K
import numpy as np
import tensorflow as tf
# tf.config.run_functions_eagerly(True)
# output.numpy()
model = Model(inputs=[w_inputs, c_inputs], outputs=output)
print(model.output)
# model.summary()
# model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [ ]:
for epoch in range(10):
  loss = 0
  for _, elem in enumerate(skip_grams):
    first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
    second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
    labels = np.array(elem[1], dtype='int32')
    X = [first_elem, second_elem]
    Y = labels
    loss += model.train_on_batch(X,Y)  
  print('Epoch :',epoch + 1, 'Loss :',loss)

In [ ]:
import gensim

In [ ]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
  f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [ ]:
# 모델 로드
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [ ]:
k=w2v.most_similar(negative=['apple'])
k
# k[0]

In [ ]:
data=[
    {
        "교과목": "과학",
        "텍스트": "식물원이나 화단에 가면 다양한 모양과 색깔의 꽃을 볼 수 있습니다.\n꽃 중에는 크고 화려한 것도 있지만 작고 밋밋하여 눈에 잘 띄지 않는 것도 있습니다.\n꽃의 생김새가 다양한 만큼 꽃이 하는 일도 다양할까요?\n꽃의 생김새와 하는 일을 알아봅시다.\n\n꽃은 식물의 종류에 따라 크기, 모양, 색깔 등이 서로 다르지만 사과꽃처럼 대부분 암술, 수술, 꽃잎, 꽃받침으로 이루어져 있습니다.\n그러나 수세미오이꽃처럼 암술, 수술, 꽃잎, 꽃받침 중 일부가 없는 것도 있습니다.\n\n꽃은 씨를 만드는 일을 합니다.\n씨를 만들려면 수술에서 만든 꽃가루를 암술로 옮겨야 합니다.\n이것을 꽃가루받이 또는 수분이라고 합니다.\n꽃가루받이는 곤충, 새, 바람, 물 등의 도움으로 이루어집니다.\n"
    },
    {
        "교과목": "과학",
        "텍스트": "꽃가루받이가 이루어지고 나면 열매가 맺힙니다.\n열매의 생김새는 식물의 종류에 따라 다양하지만 열매가 하는 일은 비슷합니다.\n열매의 생김새와 하는 일을 알아봅시다.\n\n꽃가루받이가 된 암술 속에서는 씨가 생겨 자랍니다.\n씨가 자라는 동안 씨를 싸고 있는 암술이나 꽃받침 등이 함께 자라서 열매가 됩니다.\n사과는 씨와 껍질 사이에 양분이 저장되어 있는 열매로 크고 둥근 모양입니다.\n단풍나무의 열매에는 날개가 달려 있습니다.\n열매는 어린 씨를 보호하고, 씨가 익으면 멀리 퍼뜨리는 일을 합니다.\n씨를 퍼뜨리는 방법은 열매의 종류에 따라 다릅니다.\n식물이 씨를 퍼뜨리는 다양한 방법을 조사해 봅시다."
    },
    {
        "교과목": "사회",
        "텍스트": "자정 능력: 미생물이 오염 물질을 분해하여 원래의 깨끗한 상태로 되돌리는 능력이다."
    },
    {
        "교과목": "사회",
        "텍스트": "남획으로 줄어드는 어족 자원: 참치는 현대식 어업을 통해 대량으로 잡히는 생선 중 하나이다.\n선망 어선은 길이 2km, 깊이 200m정도인 대형 그물을 이용하여 참치뿐만 아니라 멸종 위기에 처한 상어, 가오리, 고래, 바다거북 등을 가리지 않고 잡아들인다.\n이른바 '혼획'이라고 하는 이 어업 방식으로 매년 많은 종류의 물고기가 멸종되고 있다.\n외래종의 유입으로 사라지는 토착종: 외래종의 유입은 자연스럽게 발생할 수 있으나, 인위적인 목적으로 이루어지면 심각한 위험을 불러올 수 있다.\n무분별하게 도입해 방사한 황소개구리에 의해 여러 종류의 민물고기가 멸종 위기에 처해 있는 상황과 나일 농어의 유입으로 200여 종의 물고기가 사라진 아프리카 빅토리아호가 대표적인 사례이다.\n수질 오염에 따른 수상 생물 종의 감소: 농업 생산량을 늘리고자 사용하는 질소와 인산 성분 비료는 장기적으로 토양뿐만 아니라 하천과 바다를 오염시킨다.\n1860년 이후 바다로 흘러들어 간 질소의 양은 2배로 늘어났다.\n바다에 녹아든 질소는 조류 (藻類)의 이상 번식을 일으켜 산소를 고갈시키고 수상 생물을 떼죽음으로 내몬다."
    },
    {
        "교과목": "사회",
        "텍스트": "인간 개발 지수 (HDI): 각국의 교육 수준과 국민 소득, 기대 수명 등을 기준으로 발전 정도를 평가하는 지수이다."
    },
    {
        "교과목": "사회",
        "텍스트": "세계 각 지역은 부존 자원 정도, 산업화 시기, 역사적 배경 등의 영향으로 발전 수준에 차이가 난다.\n일반적으로 저개발 지역은 발전 지역보다 소득 수준이 낮으며 학교나 의료 시설 등이 부족하고 주거 환경도 열악하여 삶의 질이 낮다.\n저개발 지역들은 빈곤에 따른 여러 사회 문제를 해결하고 경제 발전을 이루어 삶의 질을 높이고자 적극적으로 노력하고 있다.\n사회 기반 시설을 확충하기 위해 외국 자본의 유치를 적극적으로 추진하고 있으며, 선진 기술의 도입으로 산업 부분에서의 생산성을 높이고 있다.\n최근에는 세계화에 따라 국가 사이의 경쟁이 심해지면서 이에 대응하려는 저개발 지역들이 경제 협력 체제를 만들고 있다.\n실제로 2009 ~ 2013년 동안 저개발 지역 국가들의 연평균 경제 성장률은 선진국들보다 높게 나타난다."
    },
    {
        "교과목": "사회",
        "텍스트": "경제 협력 체제는 여러 국가가 세계화에 대응하고 경쟁력을 높이고자 구성하는 것이다.\n특히, 단일 국가의 능력으로 선진국들의 자본, 기술과 경쟁하기 어려운 저개발 국가들은 협력 체제를 구축하여 공동으로 자원을 개발하여 수출하고, 자국의 이익에 부정적인 영향을 미치는 국가들에 함께 대응하는 노력이 이루어지고 있다.\n경제 협력 체제 회원국 간에는 수출 시 부과하는 세금을 낮추거나 없애 교류를 확대하고 이를 바탕으로 상호 보완적인 경제 발전을 추진할 수 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "국제 사회는 세계 각 지역의 불평등을 완화하고자 다양한 방면으로 노력하고 있다.\n국제 연합을 중심으로 각국 정부는 공적 개발 원조를 통한 경제적 지원뿐만 아니라 스포츠, 문화 등 다양한 분야에서 협력하고 있다.\n민간 및 개인 차원에서의 협력도 활발히 이루어지고 있다.\n특히 비정부 기구 (NGO)는 저개발 지역의 어려운 현실을 시민들에게 알리고 이들 지역 주민들을 돕는 다양한 기회를 제공한다."
    },
    {
        "교과목": "사회",
        "텍스트": "공적 개발 원조: 선진국의 정부 또는 공공 기관이 저개발국의 경제 사회 발전과 복지 증진을 주목적으로 자금이나 기술을 지원하는 제도이다."
    },
    {
        "교과목": "사회",
        "텍스트": "비정부 기구 (Non-Goverment Organization): 권력이나 이윤을 추구하지 않고 조직된 자발적인 시민 단체이다."
    },
    {
        "교과목": "사회",
        "텍스트": "2008년 말, 미국에서는 주택 가격이 하락하면서 많은 은행과 보험 회사들이 파산할 위기에 처했다.\n미국 정부는 약 7,000억 달러라는 엄청난 자금을 지원하여 이들의 파산을 막으려고 노력하였다.\n시장 경제에 가장 가깝다는 미국이지만 기업의 파산으로 인해 생겨날 문제를 막기 위해 정부가 나선 것이다."
    },
    {
        "교과목": "사회",
        "텍스트": "우리나라는 시장 경제를 바탕으로 사람들에게 경제적 의사 결정의 자유를 보장한다.\n그런데 국민 건강 보험의 경우는 가입할지 말지를 개인이 선택하게끔 하지 않고, 의무적으로 가입하도록 국가가 강제한다.\n국민의 건강 수준을 높이기 위해 정부가 시장에 개입하는 것이다."
    },
    {
        "교과목": "사회",
        "텍스트": "과거 중국은 계획 경제 체제를 실시하였다.\n국가가 공장을 소유하고 생산 품목, 생산량을 결정하였다.\n하지만 지금은 토지 거래를 제외한 대부분의 경제 활동이 개인의 자유로운 선택에 따라 이루어지고 있다.\n많은 사람이 기업을 설립하고 운영하며 이윤을 추구하고 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "어느 디자이너 형제는 비가 내려도 스케치북이 젖지 않을 만큼 튼튼하고 방수성이 좋은 가방을 만들겠다고 생각했다.\n그래서 직접 회사를 차렸는데, 이 회사는 가방의 재료로 폐천막, 자동차의 안전띠, 폐자전거의 고무 튜브 등을 활용하였다.\n많은 사람이 디자인의 독특함과 자원 재활용의 의미를 높게 평가하여서 이 가방은 전 세계에 팔려 나가고 있다"
    },
    {
        "교과목": "사회",
        "텍스트": "우리나라 여러 지역이 국내외 기업 유치에 열을 올리고 있다.\n토지를 빌려주거나 세금을 줄여 주겠다고 홍보하고 있다.\n심지어 어떤 도시에서는 자기 지역에 공장을 유치하는 데 이바지한 사람에게 상금을 지급한다는 계획도 발표하였다."
    },
    {
        "교과목": "사회",
        "텍스트": "자유 시장 경제에서 기업은 생산 활동을 담당한다.\n생산의 주체로서 소비자들이 필요로 하는 재화와 서비스를 만들어 낸다.\n또한, 기업은 이윤을 증대하기 위해 새로운 상품을 만들거나 생산 기술을 개발하는데, 이러한 노력은 기업을 발전시키고 사회 전체에 긍정적인 영향을 끼친다.\n기업은 생산 과정에서 사람들을 고용하여 일자리를 주고, 일한 대가로 임금을 지급하여 가계에 소득을 제공한다.\n따라서 기업의 생산 활동이 활발하면 고용이 증가하고 가계 소득이 늘어나 경제가 활성화된다.\n또한, 기업은 각종 세금을 납부하여 국가 재정에 이바지하기도 한다.\n오늘날에는 기업의 활동이 사회적으로 큰 영향을 미치고 있다.\n경제를 활성화하는 등 긍정적인 역할도 하지만, 기업이 눈앞의 이익만을 추구하면 환경 오염이나 노동 문제 등이 심각해진다.\n따라서 기업에 요구되는 사회적 책임이 커지고 있다.\n기업의 사회적 책임이란 기업이 사회 전체의 이익에 부합하도록 의사 결정을 하는 것을 의미한다."
    },
    {
        "교과목": "사회",
        "텍스트": "한 유명한 자동차 회사가 자동차 배기가스를 조작하여 막대한 수익을 올려 왔다.\n오염 물질을 적게 배출하면서 성능이 우수하다는 회사의 홍보를 믿고 자동차를 구매한 소비자들은 기대한 만큼 성능이 뛰어나지 않다는 것을 알게 되었다.\n또한, 이 자동차들은 환경 기준을 초과하는 막대한 배기가스를 뿜어내 환경이 소리없이 망가지고 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "기업의 사회적 책임은 일반적으로 다음과 같은 4 단계로 구분된다.\n제 1단계는 경제적인 책임으로, 이윤 극대화와 고용 창출 등이다.\n제 2단계는 법적인 책임으로, 회계의 투명성, 성실한 세금 납부, 소비자의 권익 보호 등이다.\n제 3단계는 윤리적인 책임으로, 환경· 윤리 경영, 제품 안전, 여성· 현지인· 소수 인종에 대한 공정한 대우 등을 말한다.\n제 4단계는 자선적인 책임으로, 사회 공헌 활동 또는 자선· 교육· 문화· 체육 활동 등에 대한 기업의 지원을 의미한다."
    },
    {
        "교과목": "사회",
        "텍스트": "기업가 정신이란 새로운 아이디어로 새로운 상품을 개발하고 새로운 시장을 개척하려는 기업가의 혁신적인 자세를 말한다.\n기업가 정신을 통해 기업은 더 많은 이윤을 획득하고 성장할 수 있었다.\n또한, 이 과정에서 새로운 기술과 상품이 개발되면서 사람들의 삶은 더욱 풍요로워졌다."
    },
    {
        "교과목": "사회",
        "텍스트": "인◯◯는 1996년 세계 최초로 상용 체성분 분석기를 개발하였다.\n몸속 지방량, 근육량 등을 알고 싶으면 체성분 분석기에 올라가 손잡이를 양손으로 1~2분만 잡고 있으면 된다.\n현재는 손목에 차고만 있으면, 근육량, 체지방량, 체질량 지수 등을 알려 주는 시계를 개발하여, 세계 각국에 수출하고 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "선배님, 제가 장사를 하려는데 돈이 부족합니다.\n돈 좀 꾸어 주십시오.\n그래, 자네같이 성실한 사람이라면 믿고 돈을 빌려줄 수 있지.\n얼마 후 청· 일 전쟁이 일어나 가게가 모두 불타 버렸다.\n이승훈은 빌린 돈을 제날짜에 갚을 수 없게 되었다.\n전쟁 통에 가게가 불타 버려 지금은 돈을 갚을 수 없습니다.\n기한을 연장해 주시면 꼭 갚겠습니다.\n이 난리에 내게 돈을 빌린 다른 사람들은 모두 도망쳤는데, 자네는 그러지 않았구먼.\n얼마든지 더 빌려주겠네.\n성공해서 갚게나.\n이승훈은 그 돈을 밑천으로 장사를 다시 시작하여, 당대 최고의 부자가 되었다.\n이후, 오산 학교라는 민족 학교를 세우고 조국의 독립을 위해 노력하였다."
    },
    {
        "교과목": "사회",
        "텍스트": "사람은 일생 동안 경제생활을 하게 되는데, 소득을 얻을 수 있는 기간은 제한되어 있지만 소비 생활은 평생에 걸쳐 이루어진다.\n영· 유아기에서 노년기에 걸쳐 소비 활동은 지속해서 이루어지지만, 소득은 직업을 가지게 되는 청년기부터 중· 장년기에 주로 발생한다.\n또한, 시기별로 개인의 소득과 소비 수준이 달라져 소비가 소득보다 많거나 소득이 소비보다 많은 시기가 있다.\n따라서 안정적으로 경제생활을 지속하려면 일생의 소득과 소비를 고려하여 경제생활 계획을 수립하고 실천해야 한다.\n미래의 불확실한 상황이나 노후를 대비하기 위해서는 자산을 확보하고 효율적으로 자산 관리를 해야 한다.\n특히 평균 수명이 늘어나면서 은퇴 이후 안정된 노후를 보내기 위한 자산 관리의 중요성이 더욱 커지고 있다.\n자산 관리 방법들 중 예금과 적금은 수익이 비교적 낮지만 일정한 이자 수익이 있고, 원금 손실이 적어 안전성이 높다.\n회사가 발행한 주식에 투자하면 높은 수익을 기대할 수 있지만, 원금이 손실될 우려가 크다.\n한편, 국가나 공공 기관, 기업 등이 발행한 채권에 투자하거나 보험에 가입할 수도 있다.\n자산을 관리할 때는 안전성, 수익성, 유동성을 고려해야 한다.\n안전성이란 원금이 손실되지 않는 정도, 수익성이란 이익을 얻을 가능성, 유동성이란 얼마나 쉽게 현금으로 바꿀 수 있는지를 말한다."
    },
    {
        "교과목": "사회",
        "텍스트": "경제생활을 하다 보면 소득을 잘 관리하고 합리적으로 소비하더라도 돈을 빌려야 하는 상황이 생길 수 있다.\n이때 필요한 것이 신용이다.\n신용이란 금전 거래에서 채무자가 약속된 날짜에 약속한 금액을 갚을 수 있는 능력이나 이에 대한 사회적인 믿음을 말한다.\n채무를 제때 상환하지 못하면 신용이 떨어지는데, 신용이 낮으면 금융기관에서 돈을 빌리기 어려워지고 빌린다고 해도 다른 사람보다 높은 이자를 부담하게 된다.\n현대 사회는 신용 카드 한 장만 있으면 언제라도 필요한 상품을 구매할 수 있을 정도로 개인의 신용 사용이 일상화되고 있다.\n따라서 원활한 경제생활을 하려면 자신의 신용을 꾸준히 관리해야 한다."
    },
    {
        "교과목": "사회",
        "텍스트": "우리나라에서는 현금보다 신용 카드를 더 자주 사용하는 것으로 나타났다.\n한국은행 발표에 따르면 가장 많이 이용하는 지급수단은 신용 카드로 전체의 39.7 % 를 차지했다.\n금액 기준으로도 신용 카드를 이용하는 비중이 가장 높았다.\n신용 카드 사용 확대와 더불어 신용 카드의 무분별한 사용으로 인한 문제도 많이 나타나고 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "자산 관리 방법을 선택할 때는 자산의 특성에 대한 이해가 필요하다.\n적지만 안정적으로 수익을 올리는 예금, 수익은 많지만 원금마저 잃을 수 있는 주식 등 다양한 금융 상품에 관하여 잘 알아 두어야 한다.\n안전성과 수익성을 고려하여 금융 상품을 선택해야 하며, 이때 자신이 저축이나 투자를 하는 목적과 기간을 고려하여야 한다."
    },
    {
        "교과목": "사회",
        "텍스트": "주식은 주식회사가 투자자로부터 돈을 받는 대신 투자자에게 발행한 증서인데, 주식 투자는 이 주식을 구입하는 것이다.\n주식을 가진 사람을 주주라고 하는데, 회사가 수익을 얻을 경우 주주는 배당금을 받을 수 있다.\n회사가 파산할 때는 주식 가치만큼의 투자금을 잃게 되는 위험이 따른다.\n주가는 국내외 경제 여건, 회사 실적 등 다양한 요인에 의해 매일 변동되므로, 주가가 오르면 수익을 얻을 수 있으나 반대일 경우 원금 손실의 가능성이 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "채권은 국가, 은행, 회사 등이 일반 국민으로부터 돈을 빌리기 위해 발행하는 증권이다.\n채권은 일정 기간 동안 이자를 제공하고 만기에 원금을 갚는다는 증서이기 때문에 채권에 투자하면 일정한 이자를 받을 수 있다.\n그러나 수익성이 주식보다 낮은 편이고, 채권 발행 회사가 파산하면 빌려준 돈을 받지 못할 위험도 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "펀드는 전문적인 운용 기관이 투자자들로부터 모은 자금을 주식이나 채권 등에 투자하여 그 결과를 투자자들에게 돌려주는 간접 투자 상품을 말한다.\n개인의 입장에서는 적은 돈으로도 투자할 수 있으며, 전문가가 투자를 대신해 준다는 장점을 가진다."
    },
    {
        "교과목": "사회",
        "텍스트": "우리는 필요한 것들을 시장에서 사서 씁니다.\n시장에서는 교복, 신발, 학용품은 물론 주식, 노동력 등도 거래되는데, 이러한 경제 활동은 시장 경제에서 매우 중요합니다.\n이 단원에서는 경제생활의 중심인 시장의 의미와 종류를 알아보고, 상품의 가격이 시장에서 어떻게 결정되고 변동하는지를 이해합니다.\n또한, 수요자와 공급자가 가격 변동에 어떻게 반응하는지 파악하여 시장 경제에서 가격의 중요성을 알아봅시다."
    },
    {
        "교과목": "사회",
        "텍스트": "농사를 짓는 반투족은 곡식과 철제 농기구가 풍부하다.\n수렵과 채집 생활을 하는 피그미족은 꿀과 고기가 풍부하다.\n날이 어두워지면 반투족 사람들이 피그미족과의 경계 지역에 곡식과 농기구를 갖다 놓고 돌아간다.\n그러면 피그미족 사람들은 고기와 꿀을 갖다 놓고, 반투족 사람들이 가져다 놓은 물건을 가지고 마을로 들어간다."
    },
    {
        "교과목": "사회",
        "텍스트": "시장은 재화나 서비스를 사고파는 장소를 말하지만 구체적인 장소만을 뜻하지는 않는다.\n상품을 사려는 사람과 팔려는 사람이 만나 이들 간의 상호 작용을 통해 교환과 거래가 이루어지면 시장이라고 할 수 있다.\n시장의 역할은 무엇일까?\n시장은 거래 상대방을 찾는 데 들어가는 시간과 비용을 크게 줄였다.\n시장이 없었다면 사람들은 자신이 필요한 것을 판매하는 사람을 일일이 찾아다녀야 했을 것이다.\n또한, 시장은 분업과 특화를 촉진하였고 이로 인해 시장 또한 확대되었다.\n시장이 생겨나면서 사람들은 자신이 필요로 하는 상품을 모두 다 생산할 필요가 없어졌다.\n하나의 상품을 만드는 과정을 여러 단계로 나누고 더 잘 생산할 수 있는 분야에 특화하는 과정에서 더 많은 상품이 생산되고 거래는 더욱 확대되었다.\n우리 주변의 시장은 다양한 모습으로 나타난다.\n특히 오늘날에는 이전에 볼 수 없었던 새로운 시장이 생겨나고 있으며 그 비중 또한 증가하고 있다.\n주식 시장과 인터넷을 이용하여 거래가 이루어지는 전자 상거래 시장 등이 대표적인 사례이다.\n시장은 재래시장이나 대형 마트처럼 눈에 보이는 일정한 장소를 차지하는 구체적 시장과 노동 시장, 주식 시장 같이 상품이 거래되지만 눈에 보이는 장소가 없는 추상적 시장으로 구분할 수 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "민수네 학급은 이번 체육 대회 때 학급 티셔츠를 맞춰 입기로 하였다.\n민수와 친구들은 티셔츠를 사는 일을 맡아 인터넷 쇼핑몰에서 귀엽고 개성 있는 옷을 사기로 하였다.\n단체 티셔츠를 주로 판매하는 쇼핑몰을 여러 곳 둘러 본 후 가장 마음에 드는 티셔츠를 파는 곳에서 주문하였다."
    },
    {
        "교과목": "사회",
        "텍스트": "수요일 오전, 출근길 지하철에서 김OO 씨는 스마트폰을 꺼내 자주 이용하는 모바일 쇼핑 앱에 접속했다.\n중학생인 큰딸이 부탁한 소설책과 참고서를 구매하기 위해서였다.\n책을 구매한 김OO 씨는 이어 음반 기획전이 진행 중인 것을 보고 좋아하는 가수의 음반도 주문하였다."
    },
     {
        "교과목": "과학",
        "텍스트": "재영이는 아버지와 함께 식빵을 만들기로 했습니다.\n빵 반죽을 만들면서 재영이가 궁금하게 생각한 것은 무엇인지 알아봅시다.\n\n빵 반죽을 만들려면 밀가루와 설탕이 필요하지요?\n그런데 이건 뭐예요?\n빵 반죽을 발효시키는 효모란다.\n발효요?\n김치나 요구르트처럼요?\n맞아.\n효모가 설탕을 분해하면서 기체를 만들기 때문에 빵 반죽이 부풀게 된단다.\n이것을 '발효'라고 하지.\n빵 반죽을 완성했으니 발효가 될 때까지 기다려 보자.\n자, 이제 빵 반죽이 발효됐는지 확인해 볼까?\n어! 아버지의 빵 반죽은 부풀었는데, 제 빵 반죽은 왜 부풀지 않은 걸까요?"
    },
    {
        "교과목": "과학",
        "텍스트": "1 재영이가 빵 반죽을 만들면서 궁금하게 생각한 것을 이야기해 봅시다.\n2 효모가 발효하는 데 필요한 조건을 생각해 보고, 그렇게 생각한 까닭을 이야기해 봅시다.\n3 효모가 발효하는 조건을 고려해 가설을 세워 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "재영이는 아버지와 함께 빵 반죽을 만들었고, 얼마 뒤 자신의 빵 반죽만 부풀지 않은 것을 관찰했습니다.\n재영이는 왜 자신이 만든 빵 반죽만 발효되지 않았는지 궁금해졌고, 효모가 발효하는 데 온도가 영향을 미치는지 알아보는 것을 탐구 문제로 정했습니다.\n\n왜 내가 만든 빵 반죽만 발효되지 않은 걸까?\n효모가 발효하는 데 온도가 영향을 미칠까?\n탐구할 문제를 정하고 탐구의 결과를 예상하는 것을 가설 설정이라고 합니다.\n가설은 내가 관찰한 사실이나 경험, 책에서 알게 된 내용 등을 바탕으로 세울 수 있습니다.\n가설을 세울 때에는 다음과 같은 점을 생각해야 합니다.\n탐구를 하여 알아보려는 내용이 분명하게 드러나야 합니다.\n이해하기 쉽도록 간결하게 표현해야 합니다.\n탐구를 하여 가설이 맞는지 확인할 수 있어야 합니다."
    },
    {
        "교과목": "과학",
        "텍스트": "재영이는 자신의 빵 반죽이 발효되지 않은 까닭은 빵 반죽을 차가운 곳에 두었기 때문이라고 생각했습니다.\n그래서 효모가 발효하는 데 온도가 영향을 미치는지 알아보기로 하고 다음과 같은 가설을 세웠습니다.\n효모는 차가운 곳보다 따뜻한 곳에서 더 잘 발효할 것이다.\n재영이네 모둠이 세운 가설이 맞는지 확인하려면 어떻게 실험하면 좋을지 계획을 세워 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "1 가설이 맞는지 확인하려면 어떻게 실험해야 할지 토의해 봅시다.\n효모액을 넣은 시험관을 차가운 물과 따뜻한 물에 각각 담가 보자.\n2 실험에서 다르게 해야 할 조건과 같게 해야 할 조건을 찾고, 그 방법을 정해 봅시다.\n3 실험을 하면서 관찰하거나 측정해야 할 것을 정해 봅시다.\n효모가 발효한 정도를 비교하려면 무엇을 측정해야 할까?\n4 실험에 필요한 준비물을 정하고, 실험 과정을 순서대로 정리해 봅시다.\n5 모둠 구성원이 할 역할을 정해 봅시다\n\n우리 모둠에서 세운 실험 계획을 이야기해 보고, 고쳐야 할 부분을 찾아 수정해 볼까요?"
    },
    {
        "교과목": "과학",
        "텍스트": "재영이네 모둠에서는 효모가 차가운 곳보다 따뜻한 곳에서 더 잘 발효하는지 알아보려고 다음과 같이 실험 계획을 세웠습니다.\n\n실험 계획을 세울 때에는 가설이 맞는지 확인할 수 있는 실험 방법을 생각합니다.\n그리고 실험에서 다르게 해야 할 조건과 같게 해야 할 조건을 정합니다.\n재영이네 모둠은 시험관을 담글 물의 온도만 다르게 하고, 나머지 조건은 모두 같게 변인을 통제하여 실험해야 합니다.\n만약 두 가지 이상의 조건을 다르게 하여 실험하면 그 결과가 어떤 조건 때문에 나타난 것인지 알 수 없습니다.\n실험 조건을 정하고 나면 실험하면서 관찰하거나 측정해야 할 것이 무엇인지 확인합니다.\n그리고 실험에 필요한 준비물, 실험 과정, 모둠 구성원의 역할 등을 정합니다."
    },
    {
        "교과목": "과학",
        "텍스트": "재영이네 모둠은 '효모는 차가운 곳보다 따뜻한 곳에서 더 잘 발효할 것이다.'라는 가설이 맞는지 확인하기 위해 실험을 하려고 합니다.\n그래서 효모액을 넣은 시험관 두 개를 차가운 물과 따뜻한 물에 각각 담가 시험관에서 일어나는 변화를 관찰하고, 효모액의 부피를 측정하기로 했습니다.\n실험 계획에 따라 실험을 해 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "1 비커 (50mL)에 물 20mL, 설탕 두 숟가락, 효모 두 숟가락을 넣고 유리 막대로 저어 효모액을 만듭니다.\n\n2 스포이트를 이용해 눈금이 있는 시험관 두 개에 효모액을 각각 5mL씩 넣습니다.\n\n3 비커 (500mL) 두 개에 차가운 물과 따뜻한 물을 각각 400mL씩 넣고, 가열용 시험관대를 걸칩니다.\n\n4 가열용 시험관대에 2의 시험관을 각각 담그고, 시험관에서 일어나는 변화를 관찰해 봅시다.\n\n5 15분 뒤에 시험관을 꺼내 효모액의 부피를 측정해 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "실험은 변인을 통제하면서 계획한 과정대로 진행해야 합니다.\n실험을 하는 동안 관찰하거나 측정한 내용은 바로 기록하고, 실험 결과가 예상과 다르더라도 고치거나 빼지 않습니다."
    },
    {
        "교과목": "과학",
        "텍스트": "재영이네 모둠은 효모의 발효 조건을 알아보는 실험을 하고 다음과 같이 실험 결과를 기록했습니다.\n차가운 물에 담근 시험관에서는 거품이 생기지 않는다.\n그리고 시험관의 아랫부분에 가라앉은 것이 있다.\n따뜻한 물에 담근 시험관에서는 기포가 올라온다.\n그리고 시험관의 윗부분에 거품이 생기면서 구수한 냄새가 난다.\n\n자료 변환을 하면 실험 결과를 한눈에 비교하기 쉽습니다.\n자료를 변환할 때에는 실험 결과를 잘 표현할 수 있는 방법이 무엇인지 생각해 보고, 표나 그래프, 그림 등의 형태로 변환합니다.\n표로는 많은 양의 자료를 체계적으로 정리할 수 있고, 그래프로는 실험 조건과 결과의 관계를 한눈에 알아보기 쉽게 나타낼 수 있습니다.\n그림으로는 사물의 모양이나 자연 현상을 이해하기 쉽게 표현할 수 있습니다.\n\n재영이네 모둠은 실험 결과를 그래프로 나타내기로 했습니다.\n일반적으로 막대그래프는 종류별 차이를 비교할 때 사용하고, 꺾은선그래프는 시간이나 양에 따른 변화를 나타낼 때 사용합니다.\n그래프를 그릴 때에는 가로축에 실험에서 다르게 한 조건을 나타내고, 세로축에 실험하면서 측정한 값을 나타냅니다.\n재영이네 모둠의 실험 결과를 그래프로 나타내려면 어떻게 해야 할까요?\n자료를 변환해 보고, 그 의미를 해석해 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "1 실험 결과를 어떤 그래프로 나타내면 좋을지 이야기합니다.\n차가운 물과 따뜻한 물에서 발효한 정도를 한눈에 비교하려면 막대그래프가 좋을 것 같아.\n2 그래프의 제목을 정합니다.\n3 그래프의 가로축과 세로축에 무엇을 나타내면 좋을지 이야기합니다.\n4 실험 결과를 그래프로 나타내 봅시다.\n\n1 가로축에는 실험에서 다르게 한 조건을 나타내고, 세로축에는 실험에서 측정한 결과를 나타내요.\n2 가장 큰 측정값을 나타낼 수 있도록 눈금을 정해요.\n3 가로축의 차가운 물 처음과 세로축의 5mL눈금이 만나는 곳에 점을 찍고 막대를 그려요.\n\n그래프를 보고 알 수 있는 사실을 이야기해 볼까요?"
    },
    {
        "교과목": "과학",
        "텍스트": "실험 결과를 표나 그래프, 그림 등으로 변환한 뒤에는 자료를 해석하여 그 의미를 확인합니다.\n또 실험에서 다르게 한 조건과 실험 결과 사이에는 어떤 관계가 있는지 살펴봅니다.\n자료를 해석할 때에는 실험 과정을 되짚어 볼 필요가 있습니다.\n실험 조건을 통제하여 실험했는지, 관찰 또는 측정 방법이 올바른지 생각해 봐야 합니다.\n만약 실험 과정에 문제점이 있거나 실험 방법이 바르지 않다면, 그것을 고쳐 다시 실험해야 합니다."
    },
    {
        "교과목": "과학",
        "텍스트": "재영이는 냉장고에 넣어 두었던 빵 반죽이 발효되지 않은 것을 보고, 효모가 발효하는 데 온도가 영향을 미치는지 궁금해졌습니다.\n그래서 재영이네 모둠은 '효모는 차가운 곳보다 따뜻한 곳에서 더 잘 발효할 것이다.'라는 가설을 세워 실험을 했고, 다음과 같은 결과를 얻었습니다.\n\n재영이네 모둠이 세운 가설은 실험 결과와 일치하나요?\n실험 결과를 보고 가설이 맞는지 판단한 뒤 결론을 도출해 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "1 실험 결과를 보고 가설이 맞는지 판단해 봅시다.\n\n실험 결과에서 결론을 이끌어 내 볼까요?"
    },
    {
        "교과목": "과학",
        "텍스트": "결론을 내릴 때에는 실험 결과를 보고 나의 가설이 맞는지 판단하고, 탐구 문제의 해답을 찾아 정리합니다.\n실험 결과가 나의 가설과 같다면, 이를 토대로 탐구 문제의 답을 정리해 결론을 내립니다.\n실험 결과가 나의 가설과 다르다면, 가설을 수정하여 탐구를 다시 시작해야 합니다.\n탐구를 마치면 결론을 뒷받침하는 추가 실험을 하거나, 새로운 가설을 세우고 실험을 하기도 합니다.\n또는 탐구를 한 뒤 더 알고 싶은 것 중에서 탐구 문제를 정하고 탐구를 할 수 있습니다."
    },
    {
        "교과목": "과학",
        "텍스트": "1 더 알고 싶은 것 중에서 탐구 문제를 정해 봅시다.\n효모가 발효하는 데적절한 온도는 몇 °C일까?\n설탕의 양이 많으면 효모가 더 잘 발효할까?\n2 가설을 세워 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "지구와 달은 어떻게 움직일까요?\n움직이는 사진책을 만들어 지구와 달이 움직이는 모습을 관찰해 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "1 지구와 달의 운동카드를 뜯어냅니다.\n\n2 카드를 번호 순서대로 정리하여 집게로 집습니다.\n\n3 카드를 넘기면서 지구가 움직이는 모습을 관찰해 봅시다.\n또 반대로 뒤집어 카드를 넘기면서 지구와 달이 움직이는 모습을 관찰해 봅시다.\n사진책 속의 지구와 달은 어떻게 움직이는지 이야기해 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "빠르게 달리는 기차 안에서 창밖을 보면 나무나 집이 기차가 달리는 방향의 반대 방향으로 움직이는 것처럼 보입니다.\n이와 비슷한 현상이 지구에서 보는 천체의 움직임에서도 나타납니다.\n하루 동안 지구가 어떻게 움직이는지 알아볼까요?"
    },
    {
        "교과목": "과학",
        "텍스트": "1 지구의에서 우리나라를 찾아 우리나라의 동쪽, 서쪽, 남쪽, 북쪽에 붙임딱지를 붙입니다.\n2 우리나라 위에 관측자 모형이 남쪽을 향하도록 붙입니다.\n3 전등을 지구의로부터 30cm 떨어진 곳에 놓습니다.\n이때 전등을 관측자 모형의 앞쪽에 위치하도록 합니다.\n4 전등을 켜고 지구의를 서쪽에서 동쪽으로 회전시켜 봅니다.\n5 지구의가 서쪽에서 동쪽으로 회전할 때 지구의 위에 있는 관측자 모형에게는 전등이 어떻게 움직이는 것처럼 보이는지 생각해 봅시다.\n6 지구의가 회전하는 방향과 관측자 모형이 본 전등이 움직이는 방향을 비교해 봅시다.\n\n전등을 태양이라고 한다면, 5처럼 지구의 위에 있는관측자 모형에게 태양이 움직이는 것처럼 보이는 까닭은 무엇일까요?"
    },
    {
        "교과목": "과학",
        "텍스트": "지구의 북극과 남극을 이은 가상의 직선을 지구의 자전축이라고 합니다.\n지구가 자전축을 중심으로 하루에 한 바퀴씩 서쪽에서 동쪽으로 회전하는 것을 지구의 자전이라고 합니다."
    },
    {
        "교과목": "과학",
        "텍스트": "하루 동안 태양은 동쪽 하늘에서 보이기 시작하여 남쪽 하늘을 지나 서쪽 하늘로 움직이는 것처럼 보입니다.\n그러면 밤하늘에 보이는 달의 위치는 하루 동안 어떻게 달라질까요?"
    },
    {
        "교과목": "과학",
        "텍스트": "1 달을 관측하려는 장소에서 나침반을 이용하여 동쪽, 남쪽, 서쪽을 확인합니다.\n2 남쪽을 중심으로 주변 건물이나 나무 등의 위치를 표시해 봅시다.\n3 태양이 진 뒤에 같은 장소에서 일정한 시간 간격으로 관측한 달의 위치를 붙임딱지를 이용해 기록해 봅시다.\n4 달의 위치가 시간이 지남에 따라 어떻게 변하는지 확인합니다.\n\n1 하루 동안 달의 위치가 달라지는 까닭은 무엇일까요?\n2 하루 동안 태양과 달의 위치가 어떻게 달라졌는지 비교해 볼까요?"
    },
    {
        "교과목": "과학",
        "텍스트": "달도 태양과 마찬가지로 동쪽 하늘에서 남쪽 하늘을 지나 서쪽 하늘로 움직이는 것처럼 보입니다.\n그리고 밤하늘에 있는 별도 동쪽에서 서쪽으로 움직이는 것처럼 보입니다.\n이처럼 하루 동안 태양과 달, 별들의 위치가 달라지는 것처럼 보이는 까닭은 지구가 자전하기 때문입니다."
    },
    {
        "교과목": "과학",
        "텍스트": "1 광학 현미경으로 양파 표피 세포를 관찰합니다.\n2 관찰 결과를 그림과 글로 나타내 봅시다."
    },
    {
        "교과목": "과학",
        "텍스트": "식물 세포는 세포벽과 세포막으로 둘러싸여 있고 그 안에는 핵이 있습니다.\n세포는 크기와 모양이 다양하고 그에 따라 하는 일도 다릅니다.\n동물도 세포로 이루어져 있습니다.\n동물 세포에는 세포막과 핵은 있지만 식물 세포와 다르게 세포벽이 없습니다."
    },
    {
        "교과목": "과학",
        "텍스트": "식물 세포는 세포벽과 세포막으로 둘러싸여 있고 그 안에는 핵이 있습니다.\n세포는 크기와 모양이 다양하고 그에 따라 하는 일도 다릅니다.\n동물도 세포로 이루어져 있습니다.\n동물 세포에는 세포막과 핵은 있지만 식물 세포와 다르게 세포벽이 없습니다."
    },
]
texts=[]
for i in range(len(data)):
  texts.append(data[i]["텍스트"].replace("\n",""))
print(texts)  


In [ ]:
import json
file_path="./readability_subject_data.json"
k=[]
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    k=data[:]
texts=[]
for i in range(len(k)):
  texts.append(k[i]["텍스트"].replace("\n",""))

In [ ]:
kor_tokenized_doc = [text.split() for text in texts]
print(len(kor_tokenized_doc))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(kor_tokenized_doc)

In [ ]:
word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(kor_tokenized_doc)

In [ ]:
vocab_size = len(word2idx) + 1 
print(f"단어 사전의 크기 : {vocab_size}")

In [ ]:
print(encoded[0])

In [ ]:
from tensorflow.keras.preprocessing.sequence import skipgrams

In [ ]:
# 네거티브 샘플링
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:5]]
print(f"전체 샘플 수 : {len(skip_grams)}")

In [ ]:
# 첫번째 샘플인 skip_grams[0] 내 skipgrams로 형성된 데이터셋 확인
pairs, labels = skip_grams[0][0], skip_grams[0][1]

In [ ]:
# 첫번째 뉴스그룹 샘플에 대해서 생긴 pairs와 labels의 개수
print(len(pairs))
print(len(labels))

In [ ]:
print(pairs)

In [ ]:
for i in range(5):
  print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
    idx2word[pairs[i][0]], pairs[i][0], 
    idx2word[pairs[i][1]], pairs[i][1], 
    labels[i])
  )

In [ ]:
training_dataset = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded]

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

In [ ]:
embedding_dim = 100

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(vocab_size, embedding_dim)(c_inputs)

In [ ]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [ ]:
for epoch in range(10):
  loss = 0
  for _, elem in enumerate(skip_grams):
    first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
    second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
    labels = np.array(elem[1], dtype='int32')
    X = [first_elem, second_elem]
    Y = labels
    loss += model.train_on_batch(X,Y)  
  print('Epoch :',epoch + 1, 'Loss :',loss)

In [ ]:
f = open('kor_vectors.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
  f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [ ]:
# 모델 로드
import re
import random
w2v = gensim.models.KeyedVectors.load_word2vec_format('./kor_vectors.txt', binary=False)
sep_sentence=[]
res=[]
res_ans=[]
for i in range(len(texts)):
  # print(texts[i])
  cur_text=texts[i]
  # print(cur_text.split("."))
  first_split_list=cur_text.split(".")
  
  for j in range(len(first_split_list)):
    if first_split_list[j] != '':
      sep_sentence.append(first_split_list[j])
# print(sep_sentence)
# print(len(sep_sentence))      
for i in range(len(sep_sentence)):
  cur_text=sep_sentence[i] # 문장 하나
  res.append(cur_text)
  res_ans.append(1)
  cur_text_words=cur_text.split(" ") # 문장을 단어별로 쪼갠 리스트
  candidate_numbers=[x for x in range(len(cur_text_words))] # 현재 문장에서 어떤 index의 단어들을 replace할 것인지
  random.shuffle(candidate_numbers) # 랜덤으로 선택하기 위해 shuffle
  flag=0
  if(len(candidate_numbers)>=5):
    selected_numbers=candidate_numbers[:5]
    for j in range(3):
      cur_word=cur_text_words[selected_numbers[j]] # 바꿀 단어
      if cur_word in w2v:
        flag=1
        t=w2v.most_similar(negative=[cur_word])
        # print(t)
        for k in range(len(t)):
          negative_word=t[k][0]
          cur_text_words[selected_numbers[j]]=negative_word

        target=""
        for m in range(len(cur_text_words)):
          target+=cur_text_words[m]
          if m != len(cur_text_words)-1:
            target+=" "
          else:
            target+="."
        res.append(target)
        res_ans.append(0)
print(len(res))        
print(len(res_ans))
from pandas import DataFrame
raw_data={'sentence': res, 'ans': res_ans}
data=DataFrame(raw_data)
data.sample(n=5)
for i in range(len(res)):
  print(res[i])

        

In [ ]:
data

In [ ]:
!pip install mxnet # 코랩 환경이기 때문에 앞에 !를 붙여야 한다.
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

In [ ]:
!pip install transformers==3.0.2

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

In [ ]:
!pip install transformers==3.0.2

In [ ]:
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW # 인공지능 모델의 초기값 지정 함수를 아담으로 지정한다.
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")

In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
# 

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data_list = []
for q, label in zip(data['sentence'], data['ans'])  :
    cur_data = []
    cur_data.append(q)
    cur_data.append(str(label))

    data_list.append(cur_data)    

In [ ]:
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("잘못된 문장")
            elif np.argmax(logits) == 1:
                test_eval.append("정확한 문장")
            

        print(">> 입력하신 내용은 " + test_eval[0] + "일 가능성이 있습니다.")

In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")